# 03 - Modélisation

**Objectif** : Entraîner un modèle pour prédire la qualité de l'eau.

---

## Questions à se poser :
1. Comment séparer les données pour évaluer correctement le modèle ?
2. Faut-il normaliser les features ? Pourquoi ?
3. Quels hyperparamètres choisir pour le Random Forest ?
4. Comment interpréter les métriques R² et RMSE ?
5. Quelles features sont les plus importantes pour chaque target ?

---

## Programme du notebook :
1. Charger les données
2. Séparer en train/test (70%/30%)
3. Normaliser les features (StandardScaler)
4. Entraîner un Random Forest pour chaque target
5. Évaluer avec R² et RMSE
6. Analyser l'importance des features
7. Préparer le code pour la soumission

---
## 1. Imports et chargement

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('..')

from src.paths import WATER_QUALITY_FILE, LANDSAT_FILE, TERRACLIMATE_FILE
from src.data.load_data import load_all
from src.config import TARGETS, BENCHMARK_FEATURES, RANDOM_SEED, TEST_SIZE

print("Imports OK!")

In [ ]:
# Charger les données
X, y, site_ids, df = load_all(
    str(WATER_QUALITY_FILE),
    str(LANDSAT_FILE),
    str(TERRACLIMATE_FILE)
)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"\nFeatures utilisées: {list(X.columns)}")
print(f"Targets: {list(y.columns)}")

---
## 2. Séparation Train / Test

On garde 30% des données pour tester notre modèle.

In [ ]:
# Séparer les données
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=TEST_SIZE,
    random_state=RANDOM_SEED
)

print(f"Train: {len(X_train)} lignes")
print(f"Test: {len(X_test)} lignes")

In [ ]:
# Normaliser les features (mettre à la même échelle)
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Features normalisées!")

---
## 3. Entraîner un modèle Random Forest

**Random Forest** = beaucoup d'arbres de décision qui "votent" ensemble.

On entraîne un modèle séparé pour chaque target.

In [ ]:
# Dictionnaire pour stocker les modèles
models = {}

# Entraîner un modèle pour chaque target
for target in TARGETS:
    print(f"\nEntraînement pour: {target}")
    
    # Créer le modèle
    model = RandomForestRegressor(
        n_estimators=100,    # 100 arbres
        max_depth=10,        # Profondeur max des arbres
        random_state=RANDOM_SEED
    )
    
    # Entraîner
    model.fit(X_train_scaled, y_train[target])
    
    # Sauvegarder
    models[target] = model
    
    print(f"  -> Modèle entraîné!")

print("\nTous les modèles sont prêts!")

---
## 4. Évaluer le modèle

On utilise deux métriques :
- **R²** : entre 0 et 1, plus c'est proche de 1, mieux c'est
- **RMSE** : erreur moyenne, plus c'est petit, mieux c'est

In [ ]:
# Évaluer chaque modèle
print("Résultats sur les données TEST :")
print("=" * 50)

for target in TARGETS:
    # Prédire
    y_pred = models[target].predict(X_test_scaled)
    y_true = y_test[target]
    
    # Calculer les métriques
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    
    print(f"\n{target}:")
    print(f"  R² = {r2:.3f}")
    print(f"  RMSE = {rmse:.2f}")

In [ ]:
# Visualiser : Prédictions vs Valeurs réelles
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, target in enumerate(TARGETS):
    ax = axes[i]
    
    y_pred = models[target].predict(X_test_scaled)
    y_true = y_test[target]
    
    ax.scatter(y_true, y_pred, alpha=0.5)
    
    # Ligne parfaite (y_pred = y_true)
    min_val = min(y_true.min(), y_pred.min())
    max_val = max(y_true.max(), y_pred.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', label='Parfait')
    
    ax.set_xlabel('Valeur réelle')
    ax.set_ylabel('Prédiction')
    ax.set_title(target)
    ax.legend()

plt.tight_layout()
plt.show()

---
## 5. Importance des features

Quelles variables sont les plus utiles pour prédire ?

In [ ]:
# Afficher l'importance des features pour chaque target
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, target in enumerate(TARGETS):
    ax = axes[i]
    
    # Récupérer les importances
    importances = models[target].feature_importances_
    feature_names = X.columns
    
    # Trier par importance
    sorted_idx = np.argsort(importances)[::-1]
    
    # Afficher
    ax.bar(range(len(importances)), importances[sorted_idx])
    ax.set_xticks(range(len(importances)))
    ax.set_xticklabels([feature_names[i] for i in sorted_idx], rotation=45, ha='right')
    ax.set_title(target)
    ax.set_ylabel('Importance')

plt.tight_layout()
plt.show()

---
## 6. Créer une soumission

Pour le challenge, on doit prédire sur de nouvelles données et créer un fichier CSV.

In [ ]:
# Exemple : créer une soumission (quand on aura les vraies données de test)

# 1. Charger les données de test du challenge
# test_df = pd.read_csv('path/to/test.csv')
# X_test_challenge = test_df[BENCHMARK_FEATURES]
# X_test_challenge_scaled = scaler.transform(X_test_challenge)

# 2. Faire les prédictions
# predictions = {}
# for target in TARGETS:
#     predictions[target] = models[target].predict(X_test_challenge_scaled)

# 3. Créer le fichier de soumission
# submission = pd.DataFrame(predictions)
# submission.to_csv('../outputs/submissions/submission.csv', index=False)

print("Le code est prêt pour créer une soumission!")

---
## 7. Résumé et prochaines étapes

### Ce qu'on a fait :
- Séparé les données en train/test
- Entraîné un Random Forest pour chaque target
- Évalué avec R² et RMSE
- Visualisé les prédictions et l'importance des features

### Pour améliorer le modèle :
1. **Plus de features** : ajouter les features créées dans le notebook 02
2. **Autres modèles** : tester LightGBM, XGBoost
3. **Tuning** : ajuster les hyperparamètres (n_estimators, max_depth...)
4. **Validation croisée** : utiliser `cross_val_score` pour une évaluation plus robuste